## Tensorflow Java Project

The objective of the project is to classify chest X-rays into Covid and Normal cases using Tensorflow in Java.

Deep Java Learning (DJL) is used to classify X-ray images in Java with the help of TensorFlow. The build automation tool used in thie project is Maven.

In [1]:
%maven ai.djl:api:0.8.0
%maven ai.djl.tensorflow:tensorflow-api:0.8.0
%maven ai.djl.tensorflow:tensorflow-engine:0.8.0
%maven ai.djl.tensorflow:tensorflow-model-zoo:0.8.0
%maven org.bytedeco:javacpp:1.5.4
%maven org.slf4j:slf4j-simple:1.7.26
%maven ai.djl.tensorflow:tensorflow-native-auto:2.3.1

In [2]:
%%loadFromPOM
<dependency>
    <groupId>com.google.protobuf</groupId>
    <artifactId>protobuf-java</artifactId>
    <version>3.8.0</version>
</dependency>

Imported all the required Deep Java Learning Packages

In [3]:
import ai.djl.inference.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.*;
import ai.djl.ndarray.*;
import ai.djl.repository.zoo.*;
import ai.djl.translate.*;
import ai.djl.training.util.*;
import ai.djl.util.*;
import java.net.*;
import java.nio.file.*;
import java.util.*;

Pre-trained model was used here, since it takes some time to train the model. The code referenced for training the model is inspired from https://www.kaggle.com/aakashnain/beating-everything-with-depthwise-convolution. So, used the https://github.com/ieee8023/covid-chestxray-dataset dataset for images and classified it as Covid and Normal.

In [4]:
var modelU = "/model/saved_model.zip"

In [5]:
class MyTranslator implements Translator<Image, Classifications> {

    private static final List<String> CLASSES = Arrays.asList("Normal", "Covid");

    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        NDManager manager = ctx.getNDManager();
        NDArray array = input.toNDArray(manager, Image.Flag.COLOR);
        array = NDImageUtils.resize(array, 224).div(255.0f);
        return new NDList(array);
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        NDArray probabilities = list.singletonOrThrow();
        return new Classifications(CLASSES, probabilities);
    }

    @Override
    public Batchifier getBatchifier() {
        return Batchifier.STACK;
    }
}

In [6]:
var image = "xray.jpg";

In [7]:
Criteria<Image, Classifications> criteria =
        Criteria.builder()
                .setTypes(Image.class, Classifications.class)
                .optModelUrls(modelU)
                .optTranslator(new MyTranslator())
                .optProgress(new ProgressBar())
                .build();
 ZooModel model = ModelZoo.loadModel(criteria);

Loading:     100% |████████████████████████████████████████|


In [8]:
Predictor<Image, Classifications> predictor = model.newPredictor();
Classifications classifications = predictor.predict(image);

classifications

[
	class: "Covid", probability: 0.63486
	class: "Normal", probability: 0.36513
]